# Packages

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from geopandas.tools import sjoin
from shapely.geometry import Point, MultiPolygon, Polygon
from shapely import wkt
%matplotlib inline
import time
import folium
import json
from copy import copy

# Import non-terrestrial WDPA polygons

In [2]:
%%time
polys = gpd.GeoDataFrame.from_file('../data/processed/Nonterrestrial_WDPA_Jan2023/Nonterrestrial_WDPA_Jan2023.shp') 
# Relative path to the file based on structure of GH repo

# Note 1: For the .shp file to download from GH, make sure you've followed the git-lfs instructions in the README.md (https://github.com/dataforgoodfr/Bloom/blob/data-science/data_science/README.md)

# Note 2 : Although the import command refers to the .shp file only, all the associated file which come with the .shp should be in same folder (.shx, .prj, .dbf)
# see: https://stackoverflow.com/questions/61436956/set-shape-restore-shx-config-option-to-yes-to-restore-or-create-it


CPU times: user 19.2 s, sys: 1.53 s, total: 20.7 s
Wall time: 21.9 s


In [3]:
polys.head()

,index,WDPAID,WDPA_PID,PA_DEF,NAME,ORIG_NAME,DESIG,DESIG_ENG,DESIG_TYPE,IUCN_CAT,...,MANG_AUTH,MANG_PLAN,VERIF,METADATAID,SUB_LOC,PARENT_ISO,ISO3,SUPP_INFO,CONS_OBJ,geometry
0,0,1.0,1,1,Diamond Reef and Salt Fish Tail Reef,Diamond Reef,Marine Reserve,Marine Reserve,National,Ia,...,Fisheries Division,Not Reported,State Verified,1807,AG-04,ATG,ATG,Not Applicable,Not Applicable,"POLYGON ((-61.82494 17.18497, -61.82497 17.184..."
1,1,2.0,2,1,Palaster Reef,Palaster Reef,Marine Reserve,Marine Reserve,National,Ia,...,Fisheries Division,Not Reported,State Verified,1807,AG-10,ATG,ATG,Not Applicable,Not Applicable,"POLYGON ((-61.74007 17.52001, -61.77174 17.526..."
2,21,27.0,27,1,Folkstone,Folkstone,Marine Reserve,Marine Reserve,National,II,...,National Conservation Commission (NCC),Not Reported,Not Reported,1867,Not Reported,BRB,BRB,Not Applicable,Not Applicable,"POLYGON ((-59.63212 13.17370, -59.63263 13.168..."
3,30,46.0,46,1,Reserva Biológica Atol Das Rocas,Reserva Biológica Atol Das Rocas,Reserva Biológica,Biological Reserve,National,Ia,...,Instituto Chico Mendes de Conservação da Biodi...,Not Reported,State Verified,1802,BR-RN,BRA,BRA,Not Applicable,Not Applicable,"POLYGON ((-33.92124 -3.83378, -33.92114 -3.833..."
4,40,57.0,57,1,Parque Nacional Do Cabo Orange,Parque Nacional Do Cabo Orange,Parque,Park,National,II,...,Instituto Chico Mendes de Conservação da Biodi...,Not Reported,State Verified,1802,BR-AP,BRA,BRA,Not Applicable,Not Applicable,"POLYGON ((-50.85383 2.81366, -50.85444 2.81362..."


In [4]:
polys.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### French subset

In [5]:
polys_fra = polys[polys['PARENT_ISO'].str.contains("FRA")]
polys_fra_names = polys_fra['NAME'].values
polys_fra_geoms = polys_fra['geometry'].values

# Create test points DFs, with points inside and outside of WDPAs

### Some positive points inside Bassin d'Arcachon

In [6]:
pts_test = gpd.GeoDataFrame({'ship_name':['arc1','arc2','arc3','gir_inland','canarias1'],
                             'timestamp':['1066-01-01 01:01'] * 5 },
                            geometry=gpd.points_from_xy([-1.130158,
                                                        -1.146638,
                                                        -1.223542,
                                                        0.905394,
                                                        -16.103645,],
                                                        [44.696739,
                                                        44.729920, 
                                                        44.626411,
                                                        43.638630,
                                                        28.460471,]),
                            crs='epsg:4326')

# arc1, arc2, arc3 are in Bassin d'Arcachon
# gir_inland is not inside the Bassing d'Arcachon (random point in Gironde)

### Points from Marion/Ronan CSV 

In [7]:
pts_bloom = pd.read_csv('../../bloom_scrap.csv') 
# Relative path to the file based on structure of GH repo

In [8]:
# Turn into geopandas dataframe
pts_bloom = gpd.GeoDataFrame(
    pts_bloom, geometry=gpd.points_from_xy(pts_bloom.Longitude, pts_bloom.Latitude),
    crs = 4326 # set the coordinate system to ESPG:4326, same as `poly`
)

In [9]:
pts_bloom.head()

,timestamp,ship_name,IMO,Time of latest position,Latitude,Longitude,geometry
0,2023-01-05 10:57,AFRIKA,9175834,2023-01-05 08:51 UTC,60.08667,-3.491793,POINT (-3.49179 60.08667)
1,2023-01-05 10:58,AFRIKA,9175834,2023-01-05 08:51 UTC,60.08667,-3.491793,POINT (-3.49179 60.08667)
2,2023-01-05 11:10,AFRIKA,9175834,2023-01-05 08:51 UTC,60.08667,-3.491793,POINT (-3.49179 60.08667)
3,2023-01-05 11:19,AFRIKA,9175834,2023-01-05 10:11 UTC,60.02838,-3.606985,POINT (-3.60698 60.02838)
4,2023-01-05 11:27,AFRIKA,9175834,2023-01-05 10:11 UTC,60.02838,-3.606985,POINT (-3.60698 60.02838)


##### Adding an artificial positive points to pts_bloom

In [10]:
add_row = copy(pts_bloom.iloc[[0]])
add_row['ship_name'] = 'ARTIFICIAL'
add_row['geometry'] = pts_test['geometry'][0]
add_row['timestamp'] = pts_test['timestamp'][0]

pts_bloom_plus_pos = gpd.GeoDataFrame(pd.concat([pts_bloom, add_row]), 
                                      crs = 4326 # set the coordinate system to ESPG:4326, same as `poly`
                                      ) 


# Check whether a (set of) points are in a (set of) polygons

Note: the `sjoin` (and other join) methods require an up to date version of geopandas. 
See https://stackoverflow.com/questions/70393202/why-is-my-geopandas-installation-missing-sjoin-attributeerror-geodataframe-o:
"old version of geopandas.... github.com/geopandas/geopandas/releases requires v0.10.0+"

I am only showcasing here the very simple and efficient vectorized "join" method suggested by Samuel.
(https://data-for-good.slack.com/archives/C04MVJS7E3Z/p1676913949126249?thread_ts=1676830741.313179&cid=C04MVJS7E3Z)
(https://geopandas.org/en/stable/gallery/spatial_joins.html)

For reference, the basic non-vectorized `shapely` method to check whether a polygon contains a point is `contain`.
See https://stackoverflow.com/a/36400130/14095529.

In [11]:
match_test_fra = pts_test.sjoin(polys_fra, how="left")

In [12]:
match_test = pts_test.sjoin(polys, how="left")

In [13]:
match_bloom_fra =  pts_bloom.sjoin(polys_fra, how="left")

In [14]:
match_bloom = pts_bloom.sjoin(polys, how="left")

In [15]:
match_bloom_plus_fra = pts_bloom_plus_pos.sjoin(polys_fra, how="left")

In [16]:
match_bloom_plus = pts_bloom_plus_pos.sjoin(polys, how="left")

In [20]:
match_test[['ship_name', 'timestamp', 'geometry', 'NAME']]

,ship_name,timestamp,geometry,NAME
0,arc1,1066-01-01 01:01,POINT (-1.13016 44.69674),Bassin d'Arcachon - Secteur du delta de la Leyre
0,arc1,1066-01-01 01:01,POINT (-1.13016 44.69674),Bassin D'Arcachon
0,arc1,1066-01-01 01:01,POINT (-1.13016 44.69674),Bassin d'Arcachon et Cap Ferret
0,arc1,1066-01-01 01:01,POINT (-1.13016 44.69674),Bassin d'Arcachon et banc d'Arguin
0,arc1,1066-01-01 01:01,POINT (-1.13016 44.69674),Bassin d'Arcachon et Cap Ferret
1,arc2,1066-01-01 01:01,POINT (-1.14664 44.72992),Bassin D'Arcachon
1,arc2,1066-01-01 01:01,POINT (-1.14664 44.72992),Bassin d'Arcachon et Cap Ferret
1,arc2,1066-01-01 01:01,POINT (-1.14664 44.72992),Bassin d'Arcachon et banc d'Arguin
1,arc2,1066-01-01 01:01,POINT (-1.14664 44.72992),Bassin d'Arcachon et Cap Ferret
2,arc3,1066-01-01 01:01,POINT (-1.22354 44.62641),Bassin D'Arcachon


In [19]:
match_bloom_plus[['ship_name', 'timestamp', 'geometry', 'NAME']]

,ship_name,timestamp,geometry,NAME
0,AFRIKA,2023-01-05 10:57,POINT (-3.49179 60.08667),NaN
1,AFRIKA,2023-01-05 10:58,POINT (-3.49179 60.08667),NaN
2,AFRIKA,2023-01-05 11:10,POINT (-3.49179 60.08667),NaN
3,AFRIKA,2023-01-05 11:19,POINT (-3.60698 60.02838),NaN
4,AFRIKA,2023-01-05 11:27,POINT (-3.60698 60.02838),NaN
...,...,...,...,...
0,ARTIFICIAL,1066-01-01 01:01,POINT (-1.13016 44.69674),Bassin d'Arcachon - Secteur du delta de la Leyre
0,ARTIFICIAL,1066-01-01 01:01,POINT (-1.13016 44.69674),Bassin D'Arcachon
0,ARTIFICIAL,1066-01-01 01:01,POINT (-1.13016 44.69674),Bassin d'Arcachon et Cap Ferret
0,ARTIFICIAL,1066-01-01 01:01,POINT (-1.13016 44.69674),Bassin d'Arcachon et banc d'Arguin
